In [1]:
import sys, os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import time

### Use NetworkX graphic package to create a signed network
import networkx as nx

In [11]:
# Import data

dataset = 'sp100' 

os.chdir("../data_modified")

corr_tensor = np.load('%s_corr.npy' % (dataset))
dates = np.load('%s_dates.npy' % (dataset))
nodes = np.load('%s_nodes.npy' % (dataset))

num_examples = corr_tensor.shape[0] #number of dates
dim = corr_tensor.shape[1] #number of asset

In [6]:
# Helper Functions

def make_graph(corr_mat, node_labels, graph_type):

    G = nx.Graph()
    G.add_nodes_from(node_labels)
    dim = corr_mat.shape[0]

    if not dim == len(node_labels):
        raise ValueError('number node labels not = corr matrix dimensions')

    if graph_type=='signed':
        for i in range(dim):
            for j in range(i+1, dim):
                if corr_mat[i,j] < 0:
                    G.add_edge(node_labels[i], node_labels[j], sign=-1)
                elif corr_mat[i,j] > 0:
                    G.add_edge(node_labels[i], node_labels[j], sign=1)
    
    if graph_type=='corr':
        for i in range(dim):
            for j in range(i+1, dim):
                if corr_mat[i,j] != 0.0:
                    G.add_edge(node_labels[i], node_labels[j])
    
    if graph_type=='uncorr':
        for i in range(dim):
            for j in range(i+1, dim):
                if corr_mat[i,j] == 0:
                    G.add_edge(node_labels[i], node_labels[j])
    
    density = (2*G.number_of_edges())/(G.number_of_nodes()*(G.number_of_nodes() - 1))
                
    return G, density

In [8]:
corr_mat = corr_tensor[int(num_examples/2), :, :].copy()
        
corr_mat[(corr_mat > -1*0.3) & (corr_mat < 0.3)] = 0
G, density = make_graph(corr_mat, nodes, 'signed')

In [ ]:

# Create graph for each month and calculate frustration. 

import multiprocessing
from gurobipy import *

frustration_array = []
bicoloring_array = []
date_array = []
density_array = []
threshold_array = []
time_array = []

count = 0
for i in np.arange(0.1, 1, 0.1):
    for j in range(1, int(num_examples/5)):
        
        corr_mat = corr_tensor[j*5, :, :].copy()
        corr_mat[(corr_mat > -1*i) & (corr_mat < i)] = 0
        
        G, density = make_graph(corr_mat, nodes, 'signed')

        G_classical = nx.convert_node_labels_to_integers(G)

        signedMatrix = nx.to_numpy_matrix(G_classical,weight='sign')
        unsignedMatrix = abs(signedMatrix)

        weighted_edges = nx.get_edge_attributes(G_classical, 'sign') 

        sorted_weighted_edges={}
        for (u,v) in weighted_edges:
            if u<v:
                (sorted_weighted_edges)[(u,v)] = weighted_edges[(u,v)]
            if u>v:
                (sorted_weighted_edges)[(v,u)] = weighted_edges[(u,v)]


        try:    

            order=len(signedMatrix)
            NumberOfNegative=((-1 == signedMatrix)).sum()/2
            size=int(np.count_nonzero(signedMatrix)/2)

            neighbors={}
            Degree=[]
            for u in sorted((G_classical).nodes()):
                neighbors[u] = list((G_classical)[u])
                Degree.append(len(neighbors[u]))
            unsignedDegree=Degree

            #fixing node is based on unsigned degree
            maximum_degree = max(unsignedDegree)
            [node_to_fix]=[([i for i, j in enumerate(unsignedDegree) if j == maximum_degree]).pop()]

            # Model parameters
            model = Model("UBQP")
            model.setParam(GRB.Param.Threads, multiprocessing.cpu_count())     


            # Create decision variables and update model to integrate new variables
            x=[]
            for i in range(0,order):
                x.append(model.addVar(vtype=GRB.BINARY, name='x'+str(i))) # arguments by name
            model.update()

            # Set the objective function
            OFV=0
            for (i,j) in (sorted_weighted_edges):
                OFV = OFV + (1-(sorted_weighted_edges)[(i,j)])/2 + \
                ((sorted_weighted_edges)[(i,j)])*(x[i]+x[j]-2*x[i]*x[j]) 
            model.setObjective(OFV, GRB.MINIMIZE)

            # Add constraints to the model and update model to integrate new constraints
            model.addConstr(x[node_to_fix]==1)
            model.update() 

            # Solve
            start_time = time.time()
            model.optimize()
            solveTime = time.time() - start_time 


            # Save optimal objective function values
            obj = model.getObjective()
            objectivevalue = obj.getValue()

        except Exception as err:
            print(err)
            print("Error on matrix %d with threshold %f" % (j*5, i))
        else:
            frustration_array.append(objectivevalue)
            density_array.append(density)
            #bicoloring_array.append(opt_bicoloring)
            time_array.append(solveTime)
            date_array.append(dates[j*5])
            threshold_array.append(i)


Changed value of parameter Threads to 4
   Prev: 0  Min: 0  Max: 1024  Default: 0
Optimize a model with 1 rows, 90 columns and 1 nonzeros
Model has 3643 quadratic objective terms
Variable types: 0 continuous, 90 integer (90 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+00, 8e+01]
  QObjective range [4e+00, 4e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
Found heuristic solution: objective 994.0000000
Presolve removed 1 rows and 1 columns
Presolve time: 0.07s
Presolved: 0 rows, 89 columns, 0 nonzeros
Presolved model has 3645 quadratic objective terms
Variable types: 0 continuous, 89 integer (89 binary)

Root relaxation: objective 4.765315e+02, 69 iterations, 0.01 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

H    0     0                     788.0000000 -4248.0000   639%     -    0s
     0     0  476.53145 

KeyboardInterrupt: 

Exception ignored in: 'gurobipy.logcallbackstub'
Traceback (most recent call last):
  File "C:\ProgramData\Anaconda3\lib\site-packages\ipykernel\iostream.py", line 385, in write
    if self.echo is not None:
KeyboardInterrupt


 205350 51285  686.38338   25   74  688.00000  658.20654  4.33%   3.5   75s
 218657 51924  668.06189   40   40  688.00000  660.30406  4.03%   3.5   80s
 231686 51742  662.39586   31   41  688.00000  662.39586  3.72%   3.5   85s
 242178 51223     cutoff   38       688.00000  664.13112  3.47%   3.5   90s
 255050 50089     cutoff   35       688.00000  666.20133  3.17%   3.5   95s
 271209 47814  668.77436   32   34  688.00000  668.76050  2.80%   3.5  100s
 285504 44772  671.25399   23   76  688.00000  671.25399  2.43%   3.5  105s
 300464 40232     cutoff   33       688.00000  673.95220  2.04%   3.5  110s
 315474 34631  679.69665   40   29  688.00000  676.75125  1.63%   3.5  115s
 329194 27698     cutoff   43       688.00000  679.55105  1.23%   3.5  120s


In [33]:
# Create Pandas DataFrame for results
pd.DataFrame(data={"date": date_array, "threshold": threshold_array, "density": density_array, 
                   "frustration_score": frustration_array, 
                   "bicoloring": bicoloring_array, "computation_time": time_array}).to_csv("structbal_class_%s_res.csv" % (dataset))